In [2]:
import requests
import pandas as pd
import sqlalchemy as sa
import pyodbc

## Historical Weather Data

In [3]:
locations = [
    {'Province': 'Ontario', 'City': 'North York', 'PostalCode': 'M3J1P3', 'Latitude': '43.773784', 'Longitude': '-79.500362'},
    {'Province': 'Manitoba', 'City': 'Winnipeg', 'PostalCode': 'R0G2W0', 'Latitude': '49.798866', 'Longitude': '-97.064934'},
    {'Province': 'Quebec', 'City': 'Saguenay-Lac-Saint-Jean', 'PostalCode': 'G0V0A1', 'Latitude': '48.211288', 'Longitude': '-70.069969'},
    {'Province': 'Saskatchewan', 'City': 'Regina', 'PostalCode': 'S0A0A0', 'Latitude': '50.615061', 'Longitude': '-103.412753'},
    {'Province': 'Ontario', 'City': 'Thunder Bay', 'PostalCode': 'P0M2J0', 'Latitude': '48.68475', 'Longitude': '-85.641417'},
    {'Province': 'Saskatchewan', 'City': 'Saskatoon', 'PostalCode': 'S0G0A6', 'Latitude': '51.627971', 'Longitude': '-106.439486'},
    {'Province': 'Ontario', 'City': 'Sudbury', 'PostalCode': 'P0H0B9', 'Latitude': '46.442016', 'Longitude': '-80.311475'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A7', 'Latitude': '53.825792', 'Longitude': '-113.326611'},
    {'Province': 'Nunavut', 'City': 'Baffin', 'PostalCode': 'X0A0A0', 'Latitude': '73.005278', 'Longitude': '-85.033056'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A9', 'Latitude': '53.83726', 'Longitude': '-113.412126'},
    {'Province': 'Quebec', 'City': 'Gaspésie-Îles-de-la-Madeleine', 'PostalCode': 'G0C2Y0', 'Latitude': '48.175265', 'Longitude': '-65.249235'},
    {'Province': 'Nunavut', 'City': 'Keewatin', 'PostalCode': 'X0C0H0', 'Latitude': '66.556724', 'Longitude': '-86.21719'},
    {'Province': 'Northwest Territories', 'City': 'Region 6', 'PostalCode': 'X1A1Y1', 'Latitude': '62.448206', 'Longitude': '-114.373354'}
]

current_key = '5Mm3pqxcyXVRP41KusUZyaRbB2AMDWKw' 
headers = {"accept": "application/json", 'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"} 

historical_weather_data = []

for location in locations:
    lat = location['Latitude']
    lon = location['Longitude']
    pro = location['Province']
    city = location['City']
    code = location['PostalCode']
    current_url = f'https://api.tomorrow.io/v4/weather/history/recent?location={lat},{lon}&timesteps=1d&apikey={current_key}'
    
    response = requests.get(current_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        for timeline in data['timelines']['daily']:
            weather_info = {
                'Latitude': lat,
                'Longitude': lon,
                'Province': pro,
                'City': city,
                'Postal Code': code,
                'Time(UTC)': timeline['time'],
                'Temperature': timeline['values']['temperatureApparentAvg'],
                'Max_Temperature': timeline['values']['temperatureApparentMax'],
                'Min_Temperature': timeline['values']['temperatureApparentMin'],
                'Humidity': timeline['values']['humidityAvg']
            }
            historical_weather_data.append(weather_info)
    else:
        print(f"Failed to fetch data for location ({lat}, {lon})")
        print(response.text)

weather_df = pd.DataFrame(historical_weather_data)

print(weather_df)

Failed to fetch data for location (51.627971, -106.439486)
{"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}
Failed to fetch data for location (53.83726, -113.412126)
{"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}
Failed to fetch data for location (48.175265, -65.249235)
{"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}
     Latitude    Longitude               Province                     City  \
0   43.773784   -79.500362                Ontario               North York   
1   43.773784   -79.500362                Ontario  

## Feature Engineering

In [4]:
weather_df['Temperature_Range'] = weather_df['Max_Temperature'] - weather_df['Min_Temperature']

In [5]:
weather_df['Time(UTC)'] = pd.to_datetime(weather_df['Time(UTC)'])
weather_df['Day'] = weather_df['Time(UTC)'].dt.day
weather_df['Month'] = weather_df['Time(UTC)'].dt.month
weather_df['Hour'] = weather_df['Time(UTC)'].dt.hour

In [6]:
weather_df

,Latitude,Longitude,Province,City,Postal Code,Time(UTC),Temperature,Max_Temperature,Min_Temperature,Humidity,Temperature_Range,Day,Month,Hour
0,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-01 10:00:00+00:00,27.40,33.25,21.38,65.48,11.87,1,8,10
1,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-02 10:00:00+00:00,26.34,29.16,23.19,71.44,5.97,2,8,10
2,49.798866,-97.064934,Manitoba,Winnipeg,R0G2W0,2024-08-01 11:00:00+00:00,25.11,32.55,15.38,64.83,17.17,1,8,11
3,49.798866,-97.064934,Manitoba,Winnipeg,R0G2W0,2024-08-02 11:00:00+00:00,24.96,28.03,15.38,49.20,12.65,2,8,11
4,48.211288,-70.069969,Quebec,Saguenay-Lac-Saint-Jean,G0V0A1,2024-08-01 10:00:00+00:00,23.75,29.30,19.50,76.65,9.80,1,8,10
5,48.211288,-70.069969,Quebec,Saguenay-Lac-Saint-Jean,G0V0A1,2024-08-02 10:00:00+00:00,26.04,30.76,21.00,75.38,9.76,2,8,10
6,50.615061,-103.412753,Saskatchewan,Regina,S0A0A0,2024-08-01 12:00:00+00:00,20.85,27.09,10.88,54.65,16.21,1,8,12
7,50.615061,-103.412753,Saskatchewan,Regina,S0A0A0,2024-08-02 12:00:00+00:00,25.74,31.38,12.19,41.50,19.19,2,8,12
8,48.68475,-85.641417,Ontario,Thunder Bay,P0M2J0,2024-08-01 10:00:00+00:00,24.26,30.87,17.50,61.48,13.37,1,8,10
9,48.68475,-85.641417,Ontario,Thunder Bay,P0M2J0,2024-08-02 10:00:00+00:00,26.31,29.95,17.50,47.00,12.45,2,8,10


In [7]:
# Database Connection
connection_url = sa.engine.URL.create(
    drivername = "mssql+pyodbc",
    username   = "aimerliu",
    password   = "2024!Schulich",
    host       = "mban2024-ms-sql-server.c1oick8a8ywa.ca-central-1.rds.amazonaws.com",
    port       = "1433",
    database   = "aimerliu_db",
    query = {
        "driver" : "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate" : "yes"
    }
)
my_engine = sa.create_engine(connection_url)

In [8]:
# Ingest the data into your own database in our Microsoft SQL Server
weather_df.to_sql(
    name='Final Historical Weather Data',
    con=my_engine,
    schema = 'uploads',
    if_exists='replace',
    index=False,
    dtype= {
        'Latitude': sa.types.DECIMAL(10,3),
        'Longitude': sa.types.DECIMAL(10,3),
        'Province': sa.types.VARCHAR(30),
        'City': sa.types.VARCHAR(40),
        'Postal Code': sa.types.VARCHAR(10),
        'Time(UTC)': sa.types.DATETIME,
        'Temperature': sa.types.FLOAT,
        'Max_Temperature': sa.types.FLOAT,
        'Min_Temperature': sa.types.FLOAT,
        'Humidity': sa.types.FLOAT,
        'Temperature_Range': sa.types.DECIMAL(10,3),
        'Day': sa.types.Integer,
        'Month': sa.types.Integer,
        'Hour': sa.types.Integer
    },
    method='multi'
)

20